In [28]:
import pandas as pd

df = pd.read_csv("data/data.csv")
df.head()

,SYM Im adding more every month up till the point this hits 50 SYM after that I ll start working on my SYM SYM and SYM,Bullish
0,SYM SYM SYM SYM SYM FEDs balance sheet total a...,Bearish
1,SYM SYM SYM SYM SYM X,Bullish
2,SYM going down to 300 in next 2 weeks,Bearish
3,How will this affect the market SYM price may ...,Bullish
4,SYM pos,Bullish


In [29]:
len(df)

17530

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee07befc0>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c7d5e40>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9cad200>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc19adc0>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4eea0eb580>
Retrying in 21s...
Unexpected error:Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee80aa3c0>
Retrying in 21s...
 Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc1d1f40>
Retrying in 21s...
<class 'KeyError

In [30]:
# -*- coding: utf-8 -*-
import requests
import json
from json import JSONDecodeError
import re
import datetime
from datetime import date, timedelta
import time
import sys
import threading
import random

def printToFile(linesOfData):
    with open('data/data.csv','a') as file:
        for line in linesOfData:
            file.write(line)
            file.write('\n')

def clean_tweet(tweet): 
    tweet = tweet.replace('\n',' ').replace('\r',' ')
    tweet = re.sub("\$[A-Z]*","<SYM>", tweet)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


def scrape(symbol, num_tweets=100, print_tweets=False):
    linesOfData = []
    tweets = []
    labels = []
    i = 0
    bulls = 0
    bears = 0
    page = 1
    totalTweets = 0
    
    ''' getting first page via IP rotation
        headers = {"apikey": "bf892dc0-ba87-11ea-aa7d-4f92d4ba4d88"}
        params = (("url","https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json"),);
        r = requests.get('https://app.zenscrape.com/api/v1/get', headers=headers, params=params);
    '''
    
    # getting first page without ip rotation.
    #r = requests.get('https://api.stocktwits.com/api/2/streams/symbol/' + symbol + ".json");

    jsonText = json.dumps(r.json())
    dictionary = json.loads(jsonText)
    maxi = dictionary['cursor']['max']
    length = len(dictionary['messages'])

    # number of tweets on the first page. 
    # if 30 then it works well
    #print(length)

    # master while loop can be configured 
    # either total labelled tweets or totalpages to scrape
    
    while totalTweets < num_tweets: 
            
        while i < length:
            
            isBull = 0; 
            isBear = 0; 
            isNull = 0; 
            
            message = dictionary['messages'][i]
            
            x = message['body'].replace("&#39;", "'").replace("&quot;","")
            
            x = clean_tweet(x)
            
            # dealing with the time the comment was posted. 
            strTime = message['created_at'].replace("T"," ").replace("Z","")
            dateTime = datetime.datetime.strptime(strTime, '%Y-%m-%d %H:%M:%S')
        
           
            if(str(dictionary['messages'][i]['entities']['sentiment']) == "None"):
                isNull = 0; 
        
            else:
                if str(dictionary['messages'][i]['entities']['sentiment']['basic']) == "Bullish":
                        bulls += 1
                        isBull = 1
                        totalTweets += 1
                                            
                if str(dictionary['messages'][i]['entities']['sentiment']['basic']) == "Bearish":
                        bears += 1        
                        isBear = 1 
                        totalTweets += 1
        
        
            if(isBull and len(x) > 0):
                if print_tweets:
                    print(" ---------- "+str(totalTweets)+" | Page # "+str(page)+" | Tweet # "+str(i)+" | "+strTime+" | "+message['user']['username']+" | Bullish ---------- ")
                    print(x+"\n\n")
                linesOfData.append(x+",Bullish")
                tweets.append(x)
                labels.append("Bullish")
                
            if(isBear and len(x) > 0):
                if print_tweets:
                    print(" ---------- "+str(totalTweets)+" | Page # "+str(page)+" | Tweet # "+str(i)+" | "+strTime+" | "+message['user']['username']+" | Bearish ----------")
                    print(x+"\n\n")
                linesOfData.append(x+",Bearish")
                tweets.append(x)
                labels.append("Bearish")
                
            i += 1
        
        suc = 0
        while suc < 1:
            try:
                
                # wait time for call in seconds. 
                time.sleep(0.5)
                
                ''' using the IP rotating proxy 
                headers = {"apikey": "bf892dc0-ba87-11ea-aa7d-4f92d4ba4d88"}
                params = (("url","https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json?max="+str(maxi)),);
                r = requests.get('https://app.zenscrape.com/api/v1/get', headers=headers, params=params);
                '''
                
                # call without ip rotation. 
                #r = requests.get("https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json?max="+str(maxi));
                
                jsonText = json.dumps(r.json())
                
                dictionary = json.loads(jsonText)
                
                maxi = dictionary['cursor']['max']
                
                length = len(dictionary['messages'])
                
                i = 0
                
                suc = 1
                
            except JSONDecodeError:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)

            except KeyError:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)
                                        
            except:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)
            
        page += 1
    
    # TODO: Put data in database
    print(symbol + " scraped")
    
    #print("Printing to File")
    #printToFile(linesOfData)
    #print("Completed.")
    #return pd.DataFrame(list(zip(tweets, labels)), columns=["tweets", "labels"])

def mergeNewData(dfs=[]):
    if len(dfs) > 0:
        df1 = dfs[0]
        
        for df in dfs[0:]:
            df1 = df1.append(df, ignore_index=True)
        
        return df1

Unexpected error: <class 'KeyError'>


IndentationError: unexpected indent (<ipython-input-30-9b0a1e01292e>, line 36)

Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee06b4b40>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c7d5dc0>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc1cd600>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error:Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c3a1500>
Retrying in 21s...
 'cursor'
Unexpected error: <traceback object at 0x7f4e9c082800>
Retrying in 21s...
Unexpected error:Unexpected error: <class 'KeyError'>
 Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc0b47c0>
Retrying in 21s...
<class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc723c80>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpec

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee0503b80>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee06e9cc0>
Retrying in 21s...
Unexpected error:Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4eea2d5840>
Retrying in 21s...
 <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee0061980>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9b89640>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c6d6580>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee986f580>
Retrying in 21s...
Unexpected error: <class 'KeyError

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee003cd80>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee072e200>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9b5af00>
Retrying in 21s...
Unexpected error:Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee97be600>
Retrying in 21s...
 <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee01f6680>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee8164340>
Retrying in 21s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee01c0940>
Retrying in 21s...
Unexpected error: <class 'KeyError

...

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee99309c0>
Retrying in 30s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee996bdc0>
Retrying in 30s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc4cac40>
Retrying in 30s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9b737c0>
Retrying in 30s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9a3a080>
Retrying in 30s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c730c00>
Retrying in 30s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee8615c80>
Retrying in 30s...
Unexpected error: <class 'KeyError

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee986fd40>
Retrying in 30s...
Unexpected error: Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9c5c480>
Retrying in 30s...
<class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e7c720780>
Retrying in 30s...
Unexpected error:Unexpected error:Unexpected error: <class 'KeyError'> Unexpected error: <class 'KeyError'>
Unexpected error: 
<class 'KeyError'>
 'cursor'
Unexpected error: <traceback object at 0x7f4ee018bf80>
Retrying in 30s...Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc130880>
Retrying in 30s...
<class 'KeyError'>

Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc7bb440>
Retrying in 30s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee85a5040>
Retrying in 30s.

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc0cdfc0>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc1d4f00>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc7b5700>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee98e3700>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee0394340>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4eea4f3c40>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e7c762f40>
Retrying in 33s...
Unexpected error: <class 'KeyError

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee8318240>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9ce79c0>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee99b0600>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee81f1180>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee99b0e00>
Retrying in 33s...
Unexpected error: Unexpected error:<class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee981b1c0>
Retrying in 33s...
 <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9bf8940>
Retrying in 33s...
Unexpected error: <class 'KeyError

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee018b8c0>
Retrying in 33s...

Unexpected error: <traceback object at 0x7f4ee83b5d00>
Retrying in 33s...
Unexpected error:Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee986fc40>
Retrying in 33s...
 <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee993d680>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee04f2d40>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee0192bc0>
Retrying in 33s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee03ff640>
Retrying in 33s...
Unexpected error: <class 'KeyError

In [11]:
symbols = pd.read_csv("symbols.csv")
symbols = symbols["Symbol"]
random.choice(symbols)

'BEN'

In [27]:
# threaded scraping loop
num_workers = 1

symbols_queue = symbols.tolist()


while symbols_queue:
    threads = []
    for x in range(num_workers):
        sym = symbols_queue.pop()
        t = threading.Thread(target=scrape, args=(sym,))
        threads.append(t)
    
    for t in threads:
        t.start()
    

Exception in thread Thread-508:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    Exception in thread self.run()Exception in thread Thread-516
Thread-514:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
  File "/usr/lib/python3.8/threading.py", line 870, in run
Exception in thread Thread-515:
:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)Exception in thread Thread-510:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
        self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run

self._target(*self._args, **self.


  File "/usr/lib/python3.8/threading.py", line 870, in run
KeyError    Exception in thread Thread-546:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
: Exception in thread Thread-540:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-534:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread self._target(*self._args, **self._kwargs)
Exception in thread Thread-538:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
KeyErrorThread-544Thread-539:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
        self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
'cursor'
    Exception in thread self.run()self.run()
  File "/usr/lib/python3.8/threading.py

:   File "<ipython-input-10-ee673e7e471f>", line 46, in scrape
Exception in thread Thread-584:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
Thread-573:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    Exception in thread Thread-569:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-598:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-583:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
:     self.run()
  File "/usr/lib/py

Exception in thread Thread-629:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
        Exception in thread     self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run

  File "/usr/lib/python3.8/threading.py", line 870, in run
    self.run()Exception in thread Thread-624:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-562:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
    Thread-605Exception in thread Thread-620:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
Exception in thread Thread-615:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-639

    
self.run()    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-ee673e7e471f>", line 46, in scrape
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-ee673e7e471f>", line 46, in scrape
:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    Exception in thread Thread-645:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner

  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-ee673e7e471f>", line 46, in scrape
self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-ee673e7e471f>", line 46, in scrape


    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
KeyError  File "<ipython-input-10-ee673e7e471f>", line 46, in scrape
: 

  File "/usr/lib/python3.8/thr

...

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4f00fa9840>
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee8564f80>
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee833f100>
Retrying in 15s...
Unexpected error: Unexpected error: <class 'KeyError'><class 'KeyError'>

Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc0b4540>
Retrying in 15s...
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee989fbc0>
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc1c6e00>
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee989f500>
Retrying in 15s...
Unexpected error:Unexpected error:

Unexpected error:Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c35f0c0>
Retrying in 15s...
 Unexpected error:<class 'KeyError'> <class 'KeyError'>
Unexpected error: 
'cursor'
Unexpected error: <traceback object at 0x7f4ebc3fcd00>
Retrying in 15s...
Unexpected error: 'cursor'Unexpected error:
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc4b9600>
Retrying in 15s...
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c175640>
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e7c720480>
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc1e3c80>
Retrying in 15s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpect

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc283d80>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9cbad40>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc2a9d80>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c1ea9c0>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee0713800>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee03e9900>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c5b1d00>
Retrying in 18s...
Unexpected error: <class 'KeyError

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc5bc740>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee8564340>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc5f2740>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee989fc00>
Retrying in 18s...
Unexpected error:Unexpected error: <class 'KeyError'>
 <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ebc753dc0>
Retrying in 18s...
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee8489800>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee989ff00>
Retrying in 18s...
Unexpected error:Unexpected error:

Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee9cc0a80>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4ee8442800>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c795b00>
Retrying in 18s...
Unexpected error: <class 'KeyError'>
Unexpected error: 'cursor'
Unexpected error: <traceback object at 0x7f4e9c604b80>
Retrying in 18s...
